🎨 DTA Texture Generation: PseudocodeThe goal is to find the pixel values for the small Base Texture ($\mathbf{\eta}_{adv\_b}$) that minimizes the target object's detection confidence across all sampled views.1. Setup and InitializationCode snippet

// 1. Initialize the only trainable variable
Initialise Base_Texture (eta_adv_b) with random pixel values (e.g., 16x16x3).

// 2. Set up the training infrastructure
Set Optimizer (e.g., Adam) to update Base_Texture.
Load Trained_DTN_Model (f_omega).
Load Target_Detector_Model (h_theta).
Define Transformation_Set (T) of all possible camera poses, lights, etc.

// 3. Define the EOT Loss function
FUNCTION Calculate_Attack_Loss(Detector_Output):
    Find Maximum_Car_Confidence (C_max) from Detector_Output.
    Return Average_Loss = -log(1 - C_max) across the batch.
END FUNCTION
2. The Optimization Loop (EOT)The optimization loop runs for hundreds of iterations (epochs) to refine the base texture.Code snippetFOR each optimization_step (epoch):
    // -- Step A: Sampling and Data Retrieval (The EOT Input) --
    // Sample a minibatch of scene conditions from T (Expectation Over Transformation)
    Sample Minibatch_of_Transformations (T_batch) from Transformation_Set (T).
    
    // Retrieve the corresponding data from the rendering engine for this minibatch
    Get Reference_Images (X_ref) 
    Get Segmentation_Masks (X_seg) 
    Get Original_Images (X_orig)
    
    // -- Step B: The Differentiable Forward Pass --
    START Gradient_Tracking (Tape)

        // 1. Repeated Texture Projection (Pattern Alignment)
        // Calculate Projection_Matrices (M) based on each transformation in T_batch (phi + phi_rd).
        // Apply M to Base_Texture (eta_adv_b) to create the large, projected pattern.
        Projected_Texture (eta_adv_p) = Project(Base_Texture, M_shift, M_scale, M_3DRot)
        
        // 2. Masking
        Masked_Texture (eta_adv) = Projected_Texture * Segmentation_Masks (X_seg)

        // 3. DTN Rendering
        // The DTN simulates photo-realistic rendering of the new texture.
        Rendered_Object_Image (X_adv_rendered) = Trained_DTN_Model(Reference_Images, Masked_Texture)

        // 4. Scene Reconstruction
        // Combine the new rendered object with the original scene background.
        Final_Adversarial_Image (X_adv) = X_adv_rendered + (X_orig - X_seg)

        // 5. Detection and Loss Calculation
        Detector_Output = Target_Detector_Model(Final_Adversarial_Image)
        Attack_Loss (L_atk) = Calculate_Attack_Loss(Detector_Output)

    // -- Step C: Backpropagation and Update --
    
    // Calculate the gradient of the loss with respect to the Base_Texture
    Gradient = Tape.Get_Gradient(Attack_Loss, Base_Texture)

    // Update the Base_Texture's pixel values to minimize the loss
    Optimizer.Apply_Gradient(Gradient, Base_Texture)

END FOR

// Final_Texture is the Base_Texture (eta_adv_b) after all updates.